In [1]:
import numpy as np
features = np.load('Data/resnet50-all--modern_office31.npy', allow_pickle=True)


In [11]:
features

array([array([[0.04009399, 0.11124405, 0.87550896, ..., 0.20393732, 0.4629913 ,
               0.12172365],
              [0.        , 0.05338797, 0.05549291, ..., 0.5854763 , 0.44671848,
               0.        ],
              [0.        , 0.15694532, 0.01082188, ..., 0.76134664, 0.35578296,
               0.        ],
              ...,
              [0.01364302, 0.01006648, 0.0513595 , ..., 0.10517827, 1.5931052 ,
               0.0163384 ],
              [0.00610922, 0.        , 0.8894096 , ..., 1.950121  , 0.51962966,
               0.        ],
              [0.6361439 , 0.03339517, 0.        , ..., 3.506126  , 0.06407397,
               0.3294362 ]], dtype=float32)                                    ,
       array([[0.01947382, 0.27382228, 0.2777681 , ..., 0.01811878, 0.3734175 ,
               0.        ],
              [0.5110047 , 0.12335701, 0.5746149 , ..., 3.515151  , 1.0342144 ,
               0.02006049],
              [0.08016294, 0.31615242, 0.01023116, ..., 0.139349

In [16]:
type(features.tolist()) == list

True

In [4]:
features.shape

(3,)

In [18]:
for i in range(3):
    print(features[i].shape
          )

(496, 2048)
(793, 2048)
(2815, 2048)


In [6]:
import pickle
with open ('Results/features_dic.pkl','rb') as file:
    dic = pickle.load(file)

In [12]:
l = list(dic.values())

In [19]:
for i in range(3):
    print(l[i].shape
          )

(349, 256)
(236, 256)
(160, 256)


In [20]:
labels = np.load('Data/labels-resnet50-all--modern_office31.npy', allow_pickle=True)

In [22]:
labels.shape

(4104, 31)

In [26]:
with open ('Results/labels_dic.pkl','rb') as file:
        lab = pickle.load(file)
labels = list(lab.values())


In [28]:
arr = np.concatenate(labels, axis = 0 )

In [29]:
arr.shape

(745, 7)

In [3]:
import os
import json
import random 
import numpy as np
import pickle

def extract_pressure_matrices(json_data):
    pressure_matrices = []
    for entry in json_data['pressureData']:
        pressure_matrix = entry["pressureMatrix"]
        pressure_matrices.append({"pressureMatrix": pressure_matrix})
    return pressure_matrices

def extract_features_from_pressure_matrices(pressure_matrices):
    
    flattened_data = [np.array(item["pressureMatrix"]).reshape(1,-1) for item in pressure_matrices]
    #print(flattened_data[0].shape)
    return np.concatenate(flattened_data, axis=0)
dic = {}
lab = {}

# Loop through subjects
for i in [6,7,8]:
    
    folder_path = f'data/Subject_{i}/Posture_data'

    # Get all files in the folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    l = []  # List to store feature arrays
     # List to store labels
    labels_list = []
    for f in files:
        x = int(f.split('.')[0][-1])  # Extract label from filename
        file_path = os.path.join(folder_path, f)
        
        # Check if the file is a JSON file
        if file_path.endswith('.json'):
            with open(file_path, 'r') as file:
                try:
                    data = json.load(file) 
                    pressure_matrices = extract_pressure_matrices(data)
                    features = extract_features_from_pressure_matrices(pressure_matrices)
                    l.append(features)
                    
                    # Create label array with 1 at index x
                    label_array = np.zeros((features.shape[0], 7))
                    label_array[:, x-1] = 1
                    labels_list.append(label_array)
                    
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {file_path}: {e}")
        else:
            print(f"Skipping non-JSON file: {file_path}")
    num = list(range(7))
    random.shuffle(num)
    
    dic[i] = np.concatenate([l[j] for j in num], axis=0)
    print(dic[i].shape)
    lab[i] = np.concatenate([labels_list[j] for j in num],axis = 0)
for i in dic.keys():
    
    index = list(range(dic[i].shape[0]))
    random.shuffle(index)
    dic[i] = dic[i][index,:]
    lab[i] = lab[i][index,:]

with open ('Results/features_dic_6789.pkl','wb') as file:
    pickle.dump(dic,file)

with open ('Results/labels_dic_6789.pkl','wb') as file:
    pickle.dump(lab,file)

(349, 256)
(236, 256)
(160, 256)


In [6]:
lab[6]

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [5]:
dic[6].shape

(349, 256)

In [10]:
for i in dic.keys():
    
    index = list(range(dic[i].shape[0]))
    random.shuffle(index)
    dic[i] = dic[i][index,:]
    lab[i] = lab[i][index,:]
    

In [12]:
with open ('Results/features_dic_6_8shuffle.pkl','wb') as file:
    pickle.dump(dic,file)

with open ('Results/labels_dic_6_8shuffle.pkl','wb') as file:
    pickle.dump(lab,file)

In [13]:
lab[6]

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [14]:
lab[7]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])